# GWAS Data Analysis Part
In this section how to data analysis is done during this study are shown. During this analysis some part of the this analysis is run on Python and rest of them is run on R studio. All the necessary libraries install to use R program on Jupyter Notebook.

In [ ]:
import os
os.chdir('~/lastversion_gwas_analysis/PermGWAS_Scripts/')


python code 

In [2]:
import pandas as pd
import numpy as np
import os
import sys
import glob

In [ ]:
pip install rpy2
%load_ext rpy2.ipython


## GWAS functions are run 
f95_gwas (stored as f95_gwas.r) is a necessary function to load genotypic files, kinship file, to run GWAS funtions (gwas.r, emma.r, plots_gwas.r are download from https://github.com/arthurkorte/GWAS), and save output gwas files and plots is written by Arthur Korte. 

In [ ]:
%%R
f95_gwas<-function(Y,n=2,X.folder='~/lastversion_gwas_analysis/F_95',K.file='~/lastversion_gwas_analysis/F_95/K_F95.rda',incl.lm=FALSE,incl.beta=FALSE,update.snps=FALSE,save.output=T,out.format='rda',generate.plot=T,pre=colnames(Y)[n]) {

load(K.file)

  for ( u in 1:6) {

  filename<-paste(X.folder,'/X_f95_',u,'.rda',sep='')

  load(filename)
  if (u==1) {
    results<-amm_gwas(Y,X,K,m=n,include.lm=incl.lm,calculate.effect.size=incl.beta,update.top_snps=update.snps)
  rm(X)
  } else {


  output<-amm_gwas(Y,X,K,m=n,include.lm=incl.lm,calculate.effect.size=incl.beta,update.top_snps=update.snps)
  results<-rbind(results,output)
  rm(X)
  }

  }

#return(results)

if (save.output==T) {
  auto<-0
  if (('rda'%in%out.format)==T) {
name1<-paste(colnames(Y)[n],pre,'gwasf95.rda',sep='_')
save(results,file=name1)
auto<-1
  }
  if (('csv'%in%out.format)==T) {
    name1b<-paste(colnames(Y)[n],pre,'gwasf95.csv',sep='_')
    write.csv(results,file=name1b,row.names=FALSE)
    auto<-1
  }
  if(auto==0){
    cat('wrong output format to save the data specified','\n')}
}
if (generate.plot==T) {
 name2<-paste(colnames(Y)[n],pre,'gwasf95.pdf',sep='_')
 name3<-paste(colnames(Y)[n],pre,sep='_')
pdf(file=name2)
plot_gwas(results,maf_or_mac=2,mac=5,name=name3)
dev.off()
}
}


In this part, phenotypic files are read and put in a list. SLURM array are used to analysis all the phenotypic values on parallele to reduce time cost. After finishing analsis, renamed all the filename  and gwas result save as phenotypic file name (GWAS_f95_slurm.r)

In [ ]:
%%R
i <- as.numeric(Sys.getenv("SLURM_ARRAY_TASK_ID"))
##Load libraries and sources that are needed for reading files and creating a list
library(tidyverse)
library(fs)
setwd("~/lastversion_gwas_analysis/F_95/")
source("gwas.r") 
source("emma.r")
source("plots_gwas.r")
source('~/lastversion_gwas_analysis/f95_gwas.r')

##Creating an empty list and put all phenotype file in this list
setwd("~/lastversion_gwas_analysis/Over_Occ/Over_Coocurrence/") #phenotype files folder 
Y <- list()
csvfiles<-list.files(pattern = "*.csv")
print(csvfiles[i])
Y[[i]] <- read.csv(file = csvfiles[i])
Y[[i]][, 1] <- as.numeric(Y[[i]][, 1])
Y[[i]][, 2] <- as.numeric(Y[[i]][, 2])

##Rewrite the same filename 
Y <- lapply(csvfiles, function(x) {
  file <- read.csv(x)
  # Get the start of filename prefix 
  prefix = gsub("_.*", "", x)
  # Get the suffix number
  suffix = gsub(".csv*", "", x)  
  colnames(file) <- paste(colnames(file), suffix, sep='_')
  return(file)
})

##Gwas is running for each phenotype file with "f95_gwas" fuction
f95_gwas(Y[[i]],pre="")


This is an alternatif if it can access SLURM scheduler it run all the phenotypic files in a loop so, using this one consume too much time (GWAS_f952.r)

In [ ]:
%%R
###Load libraries that are needed for .csv file reading and creating list
##Load libraries and sources that are needed for reading files and creating a list
library(tidyverse)
library(fs)
setwd("~/lastversion_gwas_analysis/F_95/")
source("gwas.r") 
source("emma.r")
source("plots_gwas.r")
source('~/lastversion_gwas_analysis/f95_gwas.r')

csvfiles <-
  dir(path = "~/lastversion_gwas_analysis/Over_Occ/Over_Coocurrence/",
      pattern = ".csv",
      full.names = TRUE)

##Creating an empty list and put all phenotype file in this list
Y <- list()
for (i in seq_along(csvfiles)) {
  Y[[i]] <- read.csv(file = csvfiles[i])
  Y[[i]][, 1] <- as.numeric(Y[[i]][, 1])
  Y[[i]][, 2] <- as.numeric(Y[[i]][, 2])
}

Y<-set_names(Y,csvfiles )

##Gwas is running for each phenotype file with "f95_gwas" fuction
for (i in seq_along(csvfiles)) {
  f95_gwas(Y[[i]],pre=i)
}
saveRDS(csvfiles, file="phenotype_list.Rds") #save the list 

## Counting number of rows for phenotype files: Filter length<50 and MPC<5
In this section input of GWAS phenotype files is filtered. Phenotype files have been created on the basis of premature stop codons. These files have been created according to significant over cooccurrence of the two premature stop codons by Laura Steinmann (You can find the details of this work at https://github.com/laurasteinmann/Premature_Stop_Codons). The first premature stop codon was found in all accessions, and the phenotype value was determined according to the status of the second stop codon. The phenotypic value was coded as 1 if the first and second premature stop codons were formed at the same time, and 0 if there was only the first premature stop codon. As it seen in file names table df, first and second premature stop codons separated by underscore. This names are given according to which chromosome and which location this premature stop codon occured. In filename AT3G32920_AT2G16380.csv, in this file first premature stop codon occurred located 32920 region at chromosome 3, if second premature stop codon exist in that accesion is occured located 16380 region at chromosome 2. In this given AT3G32920_AT2G16380.csv phenotype file, 88 first premature stop codon AT3G32920 were found in this region, while only 16 second premature stop codon AT2G16380 were found in this region. This means that 16 accessions have had significant over cooccurance premature stop codon.

Phenotype file_names are put into first column of dataframe called as df. Phenotype file"s cloumn names were labeled accessions id and phenotype value as mentioned on above. Phenotype values are given 1 and 0 values, if there is an SNPs in this accession it represents 1, if there are no SNPs in that accession id it is given 0. After that number of rows (phenotype_length), number of SNPs (sum of rows), and non_SNPs (number of non SNPs) are counted and put in this df dataframe as new columns.

Although these generated phenotype files were filtered and significant ones were found as a result, prefiltering steps are still important. This is because GWAS results are expected to establish a really significant relationship between SNPs and phenotype. For the remainder of the study, SNPs were termed significant over coorrunce premature stop codon. First, min_value or MPC (Minnor phenotype count, second most found phenotype counted), MPC less than 5 (min_value) are shown as 1. As a second step, the length of the phenotype files with less than 50 accessions (phe_length<50) are shown as 1. In this way, file names with min_value or phe_length 1 are determined, put in a new list and discarded as a result of the prefilter.

In [3]:
os.chdir('~/lastversion_gwas_analysis/Over_Occ/Over_Coocurrence/')
df = pd.DataFrame(columns=('File_name', 'Phenotype_length'))

for index,i in enumerate(os.listdir('.')):
    df.loc[index] = [i,len(pd.read_csv(i).index)]

array=[]
for file_to_read in glob.glob("*.csv"):
    df2 = pd.read_csv(file_to_read)
    a=df2["Phenotype"].sum()
    array.append(a)
b=pd.DataFrame(array)
df["ko"]=b

###add new column as non_SNPs, 
df["non_ko"]=df["Phenotype_length"]-df["ko"]
#f
##add new column min SNPs or non_SNPs
df['MPC'] = df[['ko','non_ko']].min(axis=1)
#f
##add new column min_value<5 or phenotype_length<50
df["MPC<5"]=np.where(df['MPC'] <5, 1, 0)
df["Phe_length<50"]=np.where(df['Phenotype_length'] <50, 1, 0)
df.loc[(df['MPC'] < 5)  | (df['Phenotype_length'] <50), 'MPC<5_or_Phe_length<50'] = 1
df.loc[(df['MPC'] >= 5) & (df['Phenotype_length'] >=50), "MPC<5_or_Phe_length<50"] = 0
df
##save as .csv file
#df.to_csv('/home/s417377/lastversion_gwas_analysis/Over_Occ/summary_file.csv', index=False)
##find the file_name min<5_or_phe_length<50==1 and discard from folder, continue to next step rest of the files

,File_name,Phenotype_length,ko,non_ko,MPC,MPC<5,Phe_length<50,MPC<5_or_Phe_length<50
0,AT3G32920_AT2G16380.csv,88,16,72,16.0,0,0,0.0
1,AT1G42680_AT5G63630.csv,135,32,103,32.0,0,0,0.0
2,AT4G00970_AT3G28610.csv,50,37,13,13.0,0,0,0.0
3,AT5G33280_AT5G03830.csv,97,70,27,27.0,0,0,0.0
4,AT1G42680_AT3G45740.csv,135,24,111,24.0,0,0,0.0
...,...,...,...,...,...,...,...,...
1123,AT2G15930_AT4G15950.csv,311,25,286,25.0,0,0,0.0
1124,AT1G01695_AT2G15930.csv,43,3,40,3.0,1,1,1.0
1125,AT5G03830_AT5G33280.csv,331,70,261,70.0,0,0,0.0
1126,AT2G16380_AT3G32920.csv,21,16,5,5.0,0,1,1.0


### Filtered files the file_name min<5_or_phe_length<50==0 
In this step , files which does not discarded are put into new dataframe and saved. After next steps it will continue with these non-discarded files. 

In [5]:
os.chdir('~/lastversion_gwas_analysis/Over_Occ/')
filtered_files=filtered_files["file_name"]
filtered_files
filtered_files.to_csv('/home/s417377/PermGWAS/input_list.csv', index=False,  header=False)
#####find the file_name min<5_or_phe_length<50==1 and discard from folder, continue to next step to rest of the files
discard_file=(df.loc[df['min<5_or_phe_length<50'] == 1])
discard_file=discard_file["file_name"]
#discard_file.to_csv('/home/s417377/lastversion_gwas_analysis/Over_Occ/discard_files.csv', index=False, header=False)
discard_file

###discard files from discardfile.csv
path = ('/home/s417377/lastversion_gwas_analysis/Over_Occ/Over_Occ_permutation/')
os.chdir(path)
with open('discard_files.csv', "r") as list_file:
    _list = list_file.read().splitlines()
    [os.remove(os.path.join(path,f)) for f in _list]

6       AT2G41710_AT4G15950.csv
7       AT1G01695_AT5G20450.csv
9       AT2G16380_AT1G29710.csv
10      AT1G01695_AT2G27340.csv
18      AT1G65110_AT2G42730.csv
                 ...           
1118    AT2G16380_AT2G41470.csv
1119    AT3G45740_AT2G16380.csv
1121    AT5G51795_AT2G04845.csv
1124    AT1G01695_AT2G15930.csv
1126    AT2G16380_AT3G32920.csv
Name: file_name, Length: 390, dtype: object

#### MAC>5  Filter
In this part, MAC (Minnor Allele Cound) value is less than 5 is discarded from insdide of the all files and saved. Minnor Allele Count is the second most found allel in a population, studies showed that it has a major affect on traits (). The rest of the steps are continue with this new filtered files (mac_maf.r).  

In [ ]:
%%R
library(tidyverse)
library(fs)
setwd("~/lastversion_gwas_analysis/Over_Occ/Over_Coocurrence/")
######filter for  mac>5
Y <- list()
GWAS_F95 <- list.files(pattern = ".rda")
dd<-grep(".rda", list.files(), value=T)
d<-data.matrix(dd)
for (i in 1:length((d))){
    yyz<-load(d[i])
    R<-subset(results, results$MAC>5 )
    write.csv(R, file=paste( d[i],"f95_small",".csv", sep=""))
}
##change files names:
folder = "~/lastversion_gwas_analysis/new/Over_Coocurrence/"
files <- list.files(folder,pattern = "f95_.*.csv",full.names = T) 
sapply(files,FUN=function(eachPath){ 
  file.rename(from=eachPath,to= sub(pattern="f95_", paste0(""),eachPath))
})

#### Stringent Bonferroni Correction

Bonferroni correction is  of the most important step of the GWAS analysing to associate true positive SNPs with trait and prevent false positive (Type I error) type error (). But in our study is done first just Bonferroni correction but the result are associated more than expected SNPs with phenotype. In our cases stringent bonferroni correction are done with pvalue smaller than, number of SNPs multipled number of files to divided 0.05. The result of stringent bonferroni filter are saved (bonfe.r). 

In [ ]:
%%R
library(tidyverse)
library(fs)
####this is bonferoni correction version more stringent 
Y <- list()
csvfiles<-list.files(pattern = "*.csv")
Y <- lapply(csvfiles, function(x) {
  file <- read.csv(x)
  return(file)
})
for (i in 1:length((csvfiles))){
  R<-subset(Y[[i]],Y[[i]][["Pval"]]<0.05/(2800000*738))
  write.csv(R, file=paste( "filtered_",csvfiles[i],".csv", sep=""))
}

#### Gene Position files


In this part, file contains gene start and stop position were added -50 kb to start position, +50 kb to stop posistion and saved. 

In [ ]:
####add values +-50kb to ara11
import pandas as pd
import numpy as np
ara11=pd.read_csv("ara11.csv")
###bu komut ile start -50kb cikarilir and stop +50kb eklenir
ara11["Start"]=ara11["Start"]-50000
ara11["Stop"]=ara11["Stop"]+50000
print(ara11)

#### Second_gene name filter

Second gene names are added as a new column called second_gene_name. It is important to determine second gene name to search cis hits for further steps. Second genes are define to be if it is an over cooccurance premature stop codon (SNPs) or not. 



In [ ]:
import pandas as pd
small_gene=pd.DataFrame(filtered_files["file_name"])
small_gene
second_gene_name=small_gene["file_name"].str.split("_",  n=1, expand=True)
second_gene_name=second_gene_name[1].str.split(".",  n=1, expand=True)
second_gene_name["second_gene_name"]=second_gene_name[0]
second_gene_name
result=pd.concat([small_gene, second_gene_name["second_gene_name"]], axis=1, ignore_index=True)
result=result.set_axis(["file_name", "second_gene_name"], axis=1)
#result.to_csv('/home/s417377/lastversion_gwas_analysis/Over_Occ/Small_gene.csv', index=False)
result
result=pd.read_csv('/home/s417377/lastversion_gwas_analysis/Over_Occ/trans_filtered_hit.csv')
result1=pd.DataFrame(result["file_name"])
result2=pd.DataFrame(result["second_gene_name"])
first_gene_name=result["file_name"].str.split("_",  n=1, expand=True)
first_gene_name
first_gene_name["first_gene_name"]=first_gene_name[0]
first_gene_name
result3=pd.concat([first_gene_name["first_gene_name"],result2], axis=1, ignore_index=True)
result4=pd.concat([result["file_name"],result3], axis=1, ignore_index=True)
result4
result4=result4.set_axis(["file_name","first_gene_name","second_gene_name"], axis=1)
#result4.to_csv('/home/s417377/lastversion_gwas_analysis/Over_Occ/first_second_gene.csv', index=False)
result4


#### Number of Cis Hits

Stringent bonferroni correction files are used to count cis and trans hits. Each stringent bonferroni corrected files are used. The second gene name is taken from the name of these files. Each SNPs position given in these files is valued according to the Start and Stop positions in the gene location file. If these SNPs positions are between the Start and Stop position, the SNPs at that position are called cis_hits, otherwise they are called trans hits. In this way, the number of cis hits for each file is subtracted from the total number of SNPs (hits) and the number of trans hits is found (cis_trans.r).

In [ ]:
%%R
##count cis hits
library(tidyverse)
library(fs)
setwd("~/lastversion_gwas_analysis/Over_Occ/stringent_correction_738/")
ara11<-read.csv("~/lastversion_gwas_analysis/ara11_50.csv")

small_gene<-read.csv("~/lastversion_gwas_analysis/Over_Occ/Small_gene.csv")

Y <- list()
csvfiles<-list.files(pattern = "*.csv")
Y <- lapply(csvfiles, function(x) {
  file <- read.csv(x)
  return(file)
})

new_list<-list()
for (i in 1:length((csvfiles))){
  b<-ara11[which(ara11$Gene==small_gene$second_gene_name[i]),]
  c<-sum(b$Start<Y[[i]][["Pos"]] & Y[[i]][["Pos"]]<b$Stop)
  new_list<-c(new_list,c)
  new_list<-as.matrix(new_list, sep=",")
  write.csv( new_list, "small_gene_cis.csv")
}
####change the folder name########
folder = "~/lastversion_gwas_analysis/PermGWAS_Scripts/deneme2//"
files <- list.files(folder,pattern = "trans_.*.csv",full.names = T) 
sapply(files,FUN=function(eachPath){ 
  file.rename(from=eachPath,to= sub(pattern="trans_", paste0(""),eachPath))
})

#### Count cis and trans hits
After counting cis hits, trans hits counted to sustraction of total hits to cis hits. New column added to the list as when the file contain cis hits it represented with 1, othervise 0.  New column added to the list as when the file contain trans hits it represented with 1, othervise 0. Last column added to the list as when both trans and cis hits founded in the same file, it represented with 1, othervise 0.

In [179]:
##add total SNPs hits, cis hits and trans hits as a new cloumns into result(Small_gene.csv) then sum only trans only cis and non hits
os.chdir('~/lastversion_gwas_analysis/Over_Occ/stringent_correction_738/')
total_hit = pd.DataFrame(columns=('file_name', 'total_hits'))

for index,i in enumerate(os.listdir('.')):
    total_hit.loc[index] = [i,len(pd.read_csv(i).index)]

total_hit=total_hit.sort_values("file_name")
total_hit
#total_hit.to_csv('/home/s417377/lastversion_gwas_analysis/Over_Occ/total_hit.csv', index=False)
result=pd.read_csv("~/lastversion_gwas_analysis/Over_Occ/Small_gene.csv")
total_hits=pd.read_csv("~/lastversion_gwas_analysis/Over_Occ/total_hit.csv")
total_hits=total_hits["total_hits"]
result["total_hit"]=total_hits

cis_hits=pd.read_csv("~/lastversion_gwas_analysis/Over_Occ/stringent_correction_738/small_gene_cis.csv")
cis_hits=cis_hits["V1"]
cis_hits
result["cis_hits"]=cis_hits
result
result["trans_hits"]=result["total_hit"]-result["cis_hits"]
result
##add new column if cis_hits>0 and trans_hits>0:
result["cis_1"]=np.where(result['cis_hits'] >0, 1, 0)
result["trans_1"]=np.where(result['trans_hits'] >0, 1, 0)
result.loc[(result['cis_hits'] > 0)  & (result['trans_hits'] >0), 'cis_trans_1'] = 1
result.loc[(result['cis_hits'] <= 0) | (result['trans_hits'] <=0), "cis_trans_1"] = 0
#result.to_csv('/home/s417377/lastversion_gwas_analysis/Over_Occ/cis_trans_hit.csv', index=False)
result

,file_name,second_gene_name,total_hit,cis_hits,trans_hits,cis_1,trans_1,cis_trans_1
0,AT1G04380_AT2G27340.csv,AT2G27340,31,6,25,1,1,1.0
1,AT1G04380_AT4G01925.csv,AT4G01925,18,18,0,1,0,0.0
2,AT1G04380_AT4G04220.csv,AT4G04220,125,117,8,1,1,1.0
3,AT1G04380_AT5G51795.csv,AT5G51795,175,41,134,1,1,1.0
4,AT1G11180_AT1G29710.csv,AT1G29710,28,27,1,1,1,1.0
...,...,...,...,...,...,...,...,...
733,AT5G66630_AT5G20450.csv,AT5G20450,139,101,38,1,1,1.0
734,AT5G66630_AT5G27110.csv,AT5G27110,250,250,0,1,0,0.0
735,AT5G66630_AT5G45150.csv,AT5G45150,385,115,270,1,1,1.0
736,AT5G66630_AT5G48350.csv,AT5G48350,56,29,27,1,1,1.0


#### Filter 
The gene AT2G15930 is not found on ara11 notation file that cause a problem to count allthe SNPs as a trans even some of them cis gits. It is inside tair10 file but all our analysis done for ara11 so file which contain as a second gene "AT2G15930" discarded to continue of this study. 

In [185]:
##files does not contain gene "AT2G15930" filtered to continue next steps
filter="AT2G15930"
filtered_files=(result.loc[result['second_gene_name'] != "AT2G15930"])
filtered_files=filtered_files.sort_values("file_name")
#filtered_files.to_csv('/home/s417377/lastversion_gwas_analysis/Over_Occ/cis_trans_filtered_hit.csv', index=False)
filtered_files

,file_name,second_gene_name,total_hit,cis_hits,trans_hits,cis_1,trans_1,cis_trans_1
0,AT1G04380_AT2G27340.csv,AT2G27340,31,6,25,1,1,1.0
1,AT1G04380_AT4G01925.csv,AT4G01925,18,18,0,1,0,0.0
2,AT1G04380_AT4G04220.csv,AT4G04220,125,117,8,1,1,1.0
3,AT1G04380_AT5G51795.csv,AT5G51795,175,41,134,1,1,1.0
4,AT1G11180_AT1G29710.csv,AT1G29710,28,27,1,1,1,1.0
...,...,...,...,...,...,...,...,...
733,AT5G66630_AT5G20450.csv,AT5G20450,139,101,38,1,1,1.0
734,AT5G66630_AT5G27110.csv,AT5G27110,250,250,0,1,0,0.0
735,AT5G66630_AT5G45150.csv,AT5G45150,385,115,270,1,1,1.0
736,AT5G66630_AT5G48350.csv,AT5G48350,56,29,27,1,1,1.0


#### Count hits 
In this section only cis, only trans, both trans and cis hits contains file, and non hit contain number of files are counted.

In [11]:
from tabulate import tabulate
result=pd.read_csv("~/lastversion_gwas_analysis/Over_Occ/cis_trans_filtered_hit.csv")

hit_sum_result=[]
cis_sum=result.iloc[1:724, 5].sum()
trans_sum=result.iloc[1:724, 6].sum()
cis_trans_sum=result.iloc[1:724, 7].sum()
only_cis=cis_sum-cis_trans_sum
only_trans=trans_sum-cis_trans_sum
non_hit=724-only_cis-only_trans-cis_trans_sum
hit_sum_result=["cis_only",only_cis],["trans_only",only_trans],["cis_trans_sum",cis_trans_sum],["non_hit",non_hit],["------------","-----------"],["total_hit",724]
hit_sum_result

col_names=["hits","hit_sums"]
print(tabulate(hit_sum_result, headers=col_names))

hits           hit_sums
-------------  -----------
cis_only       201.0
trans_only     11.0
cis_trans_sum  478.0
non_hit        34.0
------------   -----------
total_hit      724


#### Filter Trans hits
In this section only files contain trans hits are filtered and it will continue rest of the anlysis with these files. In GWAS analysis trans hits is important role on trait

In [193]:
###filter trans files 
trans_files=(result.loc[result['trans_1'] == 1])
trans_files.to_csv('~/lastversion_gwas_analysis/Over_Occ/trans_filtered_hit.csv', index=False)
trans_files

,file_name,second_gene_name,total_hit,cis_hits,trans_hits,cis_1,trans_1,cis_trans_1
0,AT1G04380_AT2G27340.csv,AT2G27340,31,6,25,1,1,1.0
2,AT1G04380_AT4G04220.csv,AT4G04220,125,117,8,1,1,1.0
3,AT1G04380_AT5G51795.csv,AT5G51795,175,41,134,1,1,1.0
4,AT1G11180_AT1G29710.csv,AT1G29710,28,27,1,1,1,1.0
5,AT1G11180_AT1G65110.csv,AT1G65110,7992,63,7929,1,1,1.0
...,...,...,...,...,...,...,...,...
716,AT5G66630_AT2G04845.csv,AT2G04845,459,155,304,1,1,1.0
718,AT5G66630_AT4G34460.csv,AT4G34460,328,272,56,1,1,1.0
719,AT5G66630_AT5G20450.csv,AT5G20450,139,101,38,1,1,1.0
721,AT5G66630_AT5G45150.csv,AT5G45150,385,115,270,1,1,1.0


### Cis hits are removed from files
After filtering trans files, still some of the files contain cis hits inside. With this step rest of the cis hits are removed from files (cis_removed.r).

In [ ]:
%%R
####filter stringent result files, only contain trans hits
####filter stringent result files, only contain trans hits
setwd("~/lastversion_gwas_analysis/Over_Occ/")
small_gene<-read.csv("~/lastversion_gwas_analysis/Over_Occ/trans_filtered_hit.csv")
ara11<-read.csv("~/lastversion_gwas_analysis/ara11_50.csv")

Y <- list()
csvfiles<-list.files(pattern = "*.csv")
Y <- lapply(csvfiles, function(x) {
  file <- read.csv(x)
  return(file)
})
for (i in 1:length((csvfiles))){
  b<-ara11[which(ara11$Gene==S$second_gene_name[i]),]
  trials<-subset(Y[[i]],!(b$Start<Y[[i]][["Pos"]] & Y[[i]][["Pos"]]<b$Stop))
  write.csv(trials, file=paste( "filtered_",csvfiles[i],".csv", sep=""))
}

### Count number of trans genes, 
SNPs in files containing trans hits are determined to belong to which genes, and added to the trans_gene list. For this, it is found on which gene the SNPs in the trans files are located in the ara11_10 file, which contains all the SNPs. In this way, if there is a gene corresponding to all SNPs, those genes are added to the list. It is found in the genes in the 10 kb window and added to the list (count_trans_genes.r).

In [ ]:
%%R
##filtering overlapping genes in 10 kb window
library(tidyverse)
SNPs<-read.csv('~/lastversion_gwas_analysis/SNPs_11_10.csv')
ara11<-read.csv("~/lastversion_gwas_analysis/ara11_50.csv")
small_gene<-pd.read_csv("~/lastversion_gwas_analysis/Over_Occ/Small_gene.csv")

A<-read.csv('~/lastversion_gwas_analysis/f95_bonf/trans_filtered_hits.csv')
S<-A[,1:5]
S$gens_trans<-NA
S$gens_trans_10kb<-NA

####apply all files see if it is any pattern
setwd("~/lastversion_gwas_analysis/Over_Occ/trans_hits/")
Trans <- list()
csvfiles<-list.files(pattern = "*filtered_")
Trans <- lapply(csvfiles, function(x) {
  file <- read.csv(x)
  return(file)
})

for (i in 1:length((csvfiles))){
  Z<-merge(Trans[[i]][,c(4,9)],SNPs[,c(5,17,18)],by='SNP')
  S[i,6]<-paste(as.character(na.omit(unique(Z$gene_ara11))),collapse=',')
  S[i,7]<-paste(unique(unlist(strsplit(as.character(na.omit(unique(Z$genes_10kb__ara11))),split=','))),collapse=',')
  cat(i,'\n')
}

a<-nchar(gsub('[^,]','',S$gens_trans))+1
df<-as.data.frame(a)
S['gene_count']<-NA
S['gene_count']<-df
write.csv(S, "~/lastversion_gwas_analysis/overlapping_result/overlapping_genes.csv")



### Duplicated Files
In this section, compared if the files G1_G2 and G2_G1 has a comman trans genes, for this first new csv files created and inside the file combination of all G1_G2 and G2_G1 are put same row after that detected trans file duplicated (overlap_genew.r).

In [ ]:
%%R
##In this section, compared if the files G1_G2 and G2_G1 has a comman trans genes, for this first new csv files created 
#and inside the file combination of all G1_G2 and G2_G1 are put same row 
#after that detected 
#trans file duplicated 
aa<- read.csv("~/lastversion_gwas_analysis/Over_Occ/first_second_gene.csv")
new_list<-vector()
for(i in 1:nrow(aa)){
  b<-aa[which(aa$first_gene_name==aa$second_gene_name[i]&aa$second_gene_name==aa$first_gene_name[i]),]
  d<- merge(b$file_name , aa$file_name[i])
  new_list<-c(new_list,d)
  new_list<-as.matrix(new_list, sep=",")
  new<-as.data.frame(new_list)
}
####duplicated_trans_file
new$duplicated = as.character(new$duplicated)
new_list<-as.numeric(new_list)
write.table(new, "~/lastversion_gwas_analysis/Over_Occ/duplicated_trans_files.csv")
colnames(new)<-c("duplicated")
row_odd<-seq_len(nrow(new_list)) %% 2
data_row_odd<- as.data.frame(data_frame(new_list[row_odd == 1, ]))
row_even<-seq_len(nrow(new_list)) %% 2
data_even_odd<- as.data.frame(data_frame(new_list[row_odd == 0, ] ) )
combine<-data_frame(data_even_odd,data_row_odd)
data_even_odd$`new_list[row_odd == 0, ]`=as.character(data_even_odd$`new_list[row_odd == 0, ]`)
data_row_odd$`new_list[row_odd == 1, ]`=as.character(data_row_odd$`new_list[row_odd == 1, ]`)
colnames(combine)<-c("file_1", "file_2")
write.csv(combine, "~/lastversion_gwas_analysis/Over_Occ/dup_trans_files.csv")
#######trans file daki genler ayni mi degil mi?
overlpp<-read.csv("overlapping_genes.csv")

new_list<-vector()
for(i in 1:nrow(overlpp)){
   a<-overlpp[which(combine$file_1[i]==overlpp$file_name),]
   b<-overlpp[which(combine$file_2[i]==overlpp$file_name),]
   d<-overlpp[a$gens_trans[i]==b$gens_trans,]
   dd<- merge(d$gens_trans , a$file_name[i])
   new_list<-c(new_list,d)
   new_list<-as.matrix(new_list, sep=",")
}

In [ ]:
%%R
library(tidyverse)
library(fs)

dup_trans_genes<-read.csv("~/lastversion_gwas_analysis/Over_Occ/first_second_gene.csv")
#trans file duplicated 
new_list<-list()
for(i in 1:nrow(aa)){
  b<-aa[which(aa$first_gene_name==aa$second_gene_name[i]&aa$second_gene_name==aa$first_gene_name[i]),]
  d<- merge(b$file_name , aa$file_name[i])
  new_list<-c(new_list,d)
  new_list<-as.matrix(new_list, sep=",")
}
new$duplicated = as.character(new$duplicated)
new_list<-as.numeric(new_list)
write.table(new, "~/lastversion_gwas_analysis/Over_Occ/duplicated_trans_files.csv")
colnames(new)<-c("duplicated")
row_odd<-seq_len(nrow(new_list)) %% 2
data_row_odd<- as.data.frame(data_frame(new_list[row_odd == 1, ]))
row_even<-seq_len(nrow(new_list)) %% 2
data_even_odd<- as.data.frame(data_frame(new_list[row_odd == 0, ] ) )
combine<-data_frame(data_even_odd,data_row_odd)
data_even_odd$`new_list[row_odd == 0, ]`=as.character(data_even_odd$`new_list[row_odd == 0, ]`)
data_row_odd$`new_list[row_odd == 1, ]`=as.character(data_row_odd$`new_list[row_odd == 1, ]`)
colnames(combine)<-c("file_1", "file_2")
write.csv(combine, "~/lastversion_gwas_analysis/Over_Occ/dup_trans_files.csv")


In [ ]:
%%R
overlpp<-read.csv("~/lastversion_gwas_analysis/overlapping_result/perm_overlapping_genes.csv")
combine<-read.csv("~/lastversion_gwas_analysis/PermGWAS_Scripts/dupli_trans_file_names.csv")
for(i in 1:nrow(overlpp)){
  a<-overlpp[which(combine$file_1[i]==overlpp$file_name),]
  b<-overlpp[which(combine$file_2[i]==overlpp$file_name),]
  w<-unlist(strsplit(a$gens_trans, ","))
  w2<-unlist(strsplit(b$gens_trans, ","))
  da<-intersect(w,w2)
  combine[i,4]<-paste(as.character(na.omit(da)),collapse=',')
  #######  d<- merge(da$gens_trans , da$gens_trans)
  #######  new_list<-c(new_list,d)
  #######  new_list<-as.matrix(new_list, sep=",")
}
write.csv(Y, "~/lastversion_gwas_analysis/overlapping_result/duplicated_overlapping_genes.csv", row.names = FALSE)


In [ ]:
import dataframe_image as dfi
dfi.convert('~/GWAS_Python_R_code.ipynb',
                to='pdf',
                use='latex',
                center_df=True,
                max_rows=30,
                max_cols=10,
                execute=False,
                save_notebook=False,
                limit=None,
                document_name=None,
                table_conversion='chrome'
                #chrome_path=None,
                #latex_command=None,
                #output_dir=None,
                )